In [0]:
# % pip install torch==0.4.1 torchvision==0.2.1 tensorboardX
# % pip install Pillow==6.2.1
# %load_ext tensorboard

In [2]:
import torch
torch.__version__
import PIL
PIL.__version__

'6.2.1'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
% pwd
% cd drive/My\ Drive/AIAproject_FittingRoom

/content/drive/My Drive/AIAproject_FittingRoom


In [2]:
! python train_cycleTryOn.py --data_list 'example_train_pair.txt'  --keep_step  20 --decay_step  20 --save_count 10 --datamode 'example' --stage 'cycleTryOn'

Namespace(batch_size=4, checkpoint='', checkpoint_dir='model_checkpoints', data_list='example_train_pair.txt', datamode='example', dataroot='data', decay_step=20, display_count=20, fine_height=256, fine_width=192, gan_mode='lsgan', gpu_ids='', grid_size=5, keep_step=20, lr=0.0001, name='cycleTryOn', radius=5, result_dir='result', save_count=10, shuffle=False, stage='cycleTryOn', tensorboard_dir='tensorboard', useSCHP=False, uselimbs=False, workers=1)
named: cycleTryOn!
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
/content/drive/My Drive/AIAproject_FittingRoom/Unet_Dilation.py:236: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
initialization method [xavier]
step:       20, total_G_loss: 10.333042, total_D_loss: 0.945382, time: 3.166
step:       40, total_G_loss: 8.458754, tot

In [0]:
from train_cycleTryOn import *

In [0]:
def get_opt():
    # basic
    parser = argparse.ArgumentParser()
    parser.add_argument("--name", default = "cycleTryOn")
    parser.add_argument("--gpu_ids", default = "")
    parser.add_argument('-j', '--workers', type=int, default=1)
    parser.add_argument('-b', '--batch_size', type=int, default=4)
    parser.add_argument("--shuffle", action='store_true', help='shuffle input data')
    parser.add_argument("--uselimbs", action='store_true', help='use lambs or not')
    parser.add_argument("--useSCHP", action='store_true', help='use SCHP for human parsing result or not')
    # dir
    parser.add_argument("--dataroot", default = "data")
    parser.add_argument("--datamode", default = "train")
    parser.add_argument("--stage", default = "GMM", choices=['GMM', 'cycleTryOn'], help='stage you want')
    parser.add_argument("--data_list", default = "train_pair.txt")
    parser.add_argument('--tensorboard_dir', type=str, default='tensorboard', help='save tensorboard infos')
    parser.add_argument('--checkpoint_dir', type=str, default='model_checkpoints', help='save checkpoint infos')
    parser.add_argument('--checkpoint', type=str, default='', help='model checkpoint for initialization')
    parser.add_argument('--result_dir', type=str, default='result', help='save result infos')
    # model condition
    parser.add_argument("--fine_width", type=int, default = 192)
    parser.add_argument("--fine_height", type=int, default = 256)
    parser.add_argument("--radius", type=int, default = 5)
    parser.add_argument("--grid_size", type=int, default = 5)
    # train 
    parser.add_argument('--lr', type=float, default=0.0001, help='initial learning rate for adam')
    parser.add_argument("--display_count", type=int, default=20)
    parser.add_argument("--save_count", type=int, default=5000)
    parser.add_argument("--keep_step", type=int, default=100000)
    parser.add_argument("--decay_step", type=int, default=100000)
    parser.add_argument("--gan_mode", type=str, default='lsgan')
    opt = parser.parse_args(args=[])
    return opt

In [0]:
# test
opt = get_opt()
opt.name = 'test'
opt.batch_size = 4
opt.keep_step = 20
opt.decay_step = 20
opt.save_count = 10
opt.data_list = 'example_train_pair.txt'
opt.datamode = 'example'
opt.uselimbs = True
opt.useSCHP = True
opt.stage = 'cycleTryOn'

In [0]:
print("named: %s!" % (opt.name))
# create dataset
train_dataset = CPDataset(opt)
# create dataloader
train_loader = CPDataLoader(opt, train_dataset)
board = SummaryWriter(log_dir=os.path.join(opt.tensorboard_dir, opt.name))
# create model & train & save the final checkpoint
if opt.stage == 'GMM':
    model = GMM(opt)
    if not opt.checkpoint =='' and os.path.exists(opt.checkpoint):
        load_checkpoint(model, opt.checkpoint)
    train_gmm(opt, train_loader, model, board)
    save_checkpoint(model, os.path.join(opt.checkpoint_dir, opt.name, 'gmm_final.pth'))


elif opt.stage == 'cycleTryOn':
    generator = cyclegan(opt)
    if not opt.checkpoint =='' and os.path.exists(opt.checkpoint):
        load_checkpoint(model, opt.checkpoint)
    train_cycleTryOn(opt, train_loader, generator, board)
    save_checkpoint(generator, os.path.join(opt.checkpoint_dir, opt.name, 'cycleTryOn.pth'))
else:
    raise NotImplementedError('Model [%s] is not implemented' % opt.stage)


print('Finished training %s, nameed: %s!' % (opt.stage, opt.name))

named: test!
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]
initialization method [xavier]


/content/drive/My Drive/AIAproject_FittingRoom/Unet_Dilation.py:236: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)


initialization method [xavier]
step:       20, total_G_loss: 10.244451, total_D_loss: 0.851701, time: 0.717
step:       40, total_G_loss: 8.702450, total_D_loss: 2.074389, time: 0.718
Finished training cycleTryOn, nameed: test!


In [0]:
# %tensorboard -h
%tensorboard -—logdir /tensorboard/test

In [0]:
LOG_DIR = '/tensorboard/test'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"